In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


載入所需要使用的所有函式庫

In [4]:
import cv2
import os
import google.colab.patches as colab
import numpy as np

Step 1 (Setting Label)

In [11]:
# 載入txt檔
txt_file = "/content/drive/MyDrive/Colab Notebooks/113-1｜數位影像處理/HW5/train_gt.txt"
# 載入train檔（圖片）
image_floder = "/content/drive/MyDrive/Colab Notebooks/113-1｜數位影像處理/HW5/共用/train"

image_paths = []
labels = []

# 000000.bmp 5
with open(txt_file, 'r') as file:  #'r' mean只讀取txt檔
  for line in file:
    parts = line
    image_paths.append(os.path.join(image_floder, parts[0:10])) # parts[0:10] 讀取前十個字元,000000.bmp
    labels.append(parts[11]) # 讀取第11個位元，就是5

# 確認label數量
print(len(labels))

# 測試是否正確
test_image = cv2.imread(image_paths[1])
colab.cv2_imshow(test_image)
print(labels[1])

60000


0


Step 2 (Setting Measure)

In [ ]:
# 將訓練圖片數據加入，決定HOG特徵提取
winSize =   (28,28) # 視窗大小28x28
blockSize = (28,28) # 28x28
blockStride = (1,1) # 一次讀取一格
cellSize =  (14,14)
nbins = 9
# 創建HOG
HOG = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

# 儲存特徵向量和標籤
n_train = len(labels) # 計算訓練樣本的數量，即標籤列表的長度(60000)
train_image_mat = np.zeros((n_train,36), dtype=np.float32) # 儲存每張圖片的 HOG 特徵向量
train_labels_mat = np.zeros((n_train),dtype=int) # 儲存每張圖片對應的標籤

In [ ]:
# 讀取每一張圖片，計算HOG，再存入訓練資料裡
for index in range(n_train):
  print(index)
  in_image = cv2.imread(image_paths[index]) # 隨index計數，讀取圖片
  # 計算特徵向量
  x_hog = HOG.compute(in_image)
  train_image_mat[index]  = x_hog
  train_labels_mat[index] = int(labels[index])

Step 3 (Setting Parameter)

*   type of the SVMs learning algorithm
*   type of kernel function
*   a TermCriteria object that sets the criterions for stopping learning.




In [ ]:
# 創建SVM
svm = cv2.ml.SVM_create()

# Set the parameters for the SVM
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 10000, 1e-6))

Step 4 (Starting Training)

In [ ]:
# 呼叫svm.train來操作模型
svm.train(train_image_mat, cv2.ml.ROW_SAMPLE, train_labels_mat)

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/HW5") # 切換到HW5資料夾
print(os.getcwd()) # 顯示資料夾
svm.save("svm_model.xml")

Prediction Using SVMs

In [ ]:
# 載入訓練好的SVM模型
svm = cv2.ml.SVM_load ("svm_model.xml");
# 進行測試